In [18]:
!pip install nose

In [19]:
!wget -c 'https://drive.google.com/uc?export=download&id=1rh6Rgjoyloyp2hPhTlOBD16yc7sc3gPy' -O data.zip
!unzip -o data.zip
!wget -c 'https://drive.google.com/file/d/1wxUuN8QILUfuRgafKbg1zB7uVoWPcMZw/view?usp=sharing' -O CBM1.txt

--2021-12-20 03:09:40--  https://drive.google.com/uc?export=download&id=1rh6Rgjoyloyp2hPhTlOBD16yc7sc3gPy
Resolving drive.google.com (drive.google.com)... 74.125.204.138, 74.125.204.102, 74.125.204.100, ...
Connecting to drive.google.com (drive.google.com)|74.125.204.138|:443... connected.
HTTP request sent, awaiting response... 302 Moved Temporarily
Location: https://doc-0c-3c-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/egqg1flcaq1b8ugecndubiip04o8oi3k/1639969725000/07504575957156967943/*/1rh6Rgjoyloyp2hPhTlOBD16yc7sc3gPy?e=download [following]
--2021-12-20 03:09:41--  https://doc-0c-3c-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/egqg1flcaq1b8ugecndubiip04o8oi3k/1639969725000/07504575957156967943/*/1rh6Rgjoyloyp2hPhTlOBD16yc7sc3gPy?e=download
Resolving doc-0c-3c-docs.googleusercontent.com (doc-0c-3c-docs.googleusercontent.com)... 74.125.23.132, 2404:6800:4008:c02::84
Connecting to doc-0c-3c-docs.googleusercontent.com (doc-0c-

In [20]:
def read_blosum62(path):
    """
    Reads in the ncbi's BLOSUM62.txt file and loads the scoring matrix
    into a dictionary.
    
    :param: path is the full path in the local filesystem at which the .txt file is located
    :return: a dictionary of dictionaries which will hold the cost of various amino acid
    substitutions as defined in BLOSUM62.
    """
    delta = {}
    with open(path, 'r') as f:
        lines = f.readlines()[6:]
        keys = lines[0].split()
        keys[-1] = '-'
        for i, line in enumerate(lines[1:]):
            delta[keys[i]] = {k : int(v) for (k,v) in zip(keys, line.split()[1:])}  
    return delta

blosum = read_blosum62('./BLOSUM62.txt')
print(blosum)

{'A': {'A': 4, 'R': -1, 'N': -2, 'D': -2, 'C': 0, 'Q': -1, 'E': -1, 'G': 0, 'H': -2, 'I': -1, 'L': -1, 'K': -1, 'M': -1, 'F': -2, 'P': -1, 'S': 1, 'T': 0, 'W': -3, 'Y': -2, 'V': 0, 'B': -2, 'Z': -1, 'X': 0, '-': -4}, 'R': {'A': -1, 'R': 5, 'N': 0, 'D': -2, 'C': -3, 'Q': 1, 'E': 0, 'G': -2, 'H': 0, 'I': -3, 'L': -2, 'K': 2, 'M': -1, 'F': -3, 'P': -2, 'S': -1, 'T': -1, 'W': -3, 'Y': -2, 'V': -3, 'B': -1, 'Z': 0, 'X': -1, '-': -4}, 'N': {'A': -2, 'R': 0, 'N': 6, 'D': 1, 'C': -3, 'Q': 0, 'E': 0, 'G': 0, 'H': 1, 'I': -3, 'L': -3, 'K': 0, 'M': -2, 'F': -3, 'P': -2, 'S': 1, 'T': 0, 'W': -4, 'Y': -2, 'V': -3, 'B': 3, 'Z': 0, 'X': -1, '-': -4}, 'D': {'A': -2, 'R': -2, 'N': 1, 'D': 6, 'C': -3, 'Q': 0, 'E': 2, 'G': -1, 'H': -1, 'I': -3, 'L': -4, 'K': -1, 'M': -3, 'F': -3, 'P': -1, 'S': 0, 'T': -1, 'W': -4, 'Y': -3, 'V': -3, 'B': 4, 'Z': 1, 'X': -1, '-': -4}, 'C': {'A': 0, 'R': -3, 'N': -3, 'D': -3, 'C': 9, 'Q': -3, 'E': -4, 'G': -3, 'H': -3, 'I': -1, 'L': -1, 'K': -3, 'M': -1, 'F': -2, 'P': -3, '

In [21]:
UP = (-1,0)
LEFT = (0, -1)
TOPLEFT = (-1, -1)
ORIGIN = (0, 0)

def traceback_global(v, w, pointers):
    i,j = len(v), len(w)
    new_v = []
    new_w = []
    while True:
        di, dj = pointers[i][j]
        if (di,dj) == LEFT:
            new_v.append('-')
            new_w.append(w[j-1])
        elif (di,dj) == UP:
            new_v.append(v[i-1])
            new_w.append('-')
        elif (di,dj) == TOPLEFT:
            new_v.append(v[i-1])
            new_w.append(w[j-1])
        i, j = i + di, j + dj
        if (i <= 0 and j <= 0):
            break
    return ''.join(new_v[::-1])+'\n'+''.join(new_w[::-1])
    


def global_align(v, w, delta):
    """
    Returns the score of the maximum scoring alignment of the strings v and w, as well as the actual alignment as 
    computed by traceback_global. 
    
    :param: v
    :param: w
    :param: delta
    """
    M = [[0 for j in range(len(w)+1)] for i in range(len(v)+1)]
    pointers = [[0 for j in range(len(w)+1)] for i in range(len(v)+1)]
    score, alignment = None, None
    
    #nw algorithm
    for i in range(len(v)+1):
      for j in range(len(w)+1):
        if i is 0 and j is 0:
          M[i][j] = 0
        else:
          deletion = None
          insertion = None
          mmatch = None
          ops = []
          if i > 0:
            deletion = M[i - 1][j] + delta['-'][v[i-1]]
            ops.append(deletion)
          if j > 0:
            insertion = M[i][j - 1] + delta['-'][w[j-1]]
            ops.append(insertion)
          if i > 0 and j > 0:
            mmatch = M[i - 1][j - 1] + delta[v[i-1]][w[j-1]]
            ops.append(mmatch)
          M[i][j] = max(ops)
          if M[i][j] is insertion:
            pointers[i][j] = LEFT
          elif M[i][j] is deletion:
            pointers[i][j] = UP
          else:
            pointers[i][j] = TOPLEFT
    score = M[len(v)][len(w)]
    
    alignment = traceback_global(v,w, pointers)
    return score, alignment

In [22]:
UP = (-1,0)
LEFT = (0, -1)
TOPLEFT = (-1, -1)
ORIGIN = (0, 0)

def traceback_banded (v, w, pointers):
    i,j = len(v), len(w)
    new_v = []
    new_w = []
    while True:
        di, dj = pointers[i][j-i]
        if (di,dj) == LEFT:
            new_v.append('-')
            new_w.append(w[j-1])
        elif (di,dj) == UP:
            new_v.append(v[i-1])
            new_w.append('-')
        elif (di,dj) == TOPLEFT:
            new_v.append(v[i-1])
            new_w.append(w[j-1])
        i, j = i + di, j + dj
        if (i <= 0 and j <= 0):
            break
    return ''.join(new_v[::-1])+'\n'+''.join(new_w[::-1])
    
def banded_align(v, w, k, delta):
    """
    Returns the score of the maximum scoring banded alignment of the strings v and w, as well as the actual alignment as 
    computed by traceback_global. 
    
    :param: v
    :param: w
    :param: k
    :param: delta
    """
    d = len(w)-len(v)
    if abs(d)+2*k > len(w):
        return banded_align_old(v, w, k, delta)
    M = [[0 for j in range(abs(d)+1+2*k)] for i in range(len(v)+1)]
    pointers = [[0 for j in range(abs(d)+1+2*k)] for i in range(len(v)+1)]
    score, alignment = None, None
    if d >= 0:
        a, b = 0, d
    else:
        a, b = d, 0

    for i in range(len(v)+1):
        for j in range(max(i+a-k,0), min(i+b+k,len(w))+1):
            jj = j - i
            if i is 0 and j is 0:
                M[i][jj] = 0
            else:
                deletion = None
                insertion = None
                mmatch = None
                ops = []
                if i > 0 and jj < b+k:
                    deletion = M[i - 1][j - (i - 1)] + delta['-'][v[i-1]]
                    ops.append(deletion)
                if j > 0 and jj > a-k:
                    insertion = M[i][j - 1 - i] + delta['-'][w[j-1]]
                    ops.append(insertion)
                if i > 0 and j > 0:
                    mmatch = M[i - 1][jj] + delta[v[i-1]][w[j-1]]
                    ops.append(mmatch)
                M[i][jj] = max(ops)
                if M[i][jj] is insertion:
                    pointers[i][jj] = LEFT
                elif M[i][jj] is deletion:
                    pointers[i][jj] = UP
                else:
                    pointers[i][jj] = TOPLEFT
    score = M[len(v)][len(w)-len(v)]
    alignment = traceback_banded(v, w, pointers)
    return score, alignment

def banded_align_old(v, w, k, delta):
    """
    Returns the score of the maximum scoring alignment of the strings v and w, as well as the actual alignment as 
    computed by traceback_global. 
    
    :param: v
    :param: w
    :param: delta
    """
    M = [[0 for j in range(len(w)+1)] for i in range(len(v)+1)]
    pointers = [[0 for j in range(len(w)+1)] for i in range(len(v)+1)]
    score, alignment = None, None
    
    #nw algorithm
    for i in range(len(v)+1):
      for j in range(len(w)+1):
        if i is 0 and j is 0:
          M[i][j] = 0
        else:
          deletion = None
          insertion = None
          mmatch = None
          ops = []
          if i > 0 and j - i < k:
            deletion = M[i - 1][j] + delta['-'][v[i-1]]
            ops.append(deletion)
          if j > 0 and i - j < k:
            insertion = M[i][j - 1] + delta['-'][w[j-1]]
            ops.append(insertion)
          if i > 0 and j > 0:
            mmatch = M[i - 1][j - 1] + delta[v[i-1]][w[j-1]]
            ops.append(mmatch)
          M[i][j] = max(ops)
          if M[i][j] is insertion:
            pointers[i][j] = LEFT
          elif M[i][j] is deletion:
            pointers[i][j] = UP
          else:
            pointers[i][j] = TOPLEFT
    score = M[len(v)][len(w)]
    
    alignment = traceback_global(v,w, pointers)
    return score, alignment

In [23]:
import time

def print_alignment(score_alignment):
    score, alignment = score_alignment
    print('Score:',score)
    print(alignment)

In [24]:
def run_alignments(seq1, seq2):
  N = 400
  start_time = time.time()
  for _ in range(N):
      global_align(seq1, seq2, blosum)
  print(f' Global Alignment x{N}: {time.time() - start_time} seconds '.center(100, '='))
  score, alignment = global_align(seq1, seq2, blosum)
  print_alignment((score, alignment))
  k = 0
  while True:
    start_time = time.time()
    for _ in range(N):
        banded_align(seq1, seq2, k, blosum)
    print()
    print(f' Banded Alignment (k={k}) x{N}: {time.time() - start_time} seconds '.center(100, '='))
    scoreb, alignmentb = banded_align(seq1, seq2, k, blosum)
    print_alignment((scoreb, alignmentb))
    if scoreb == score: break
    k += 1

In [25]:
s1 = "LYYKCGGWHDGKPWTGCTQCVKGAKCVVQNEWY"
s2 = "IYHQCGGSKSAYPDGSLSCASGSKCVKMNDYY"
run_alignments(s1, s2)

======================== Global Alignment x400: 0.9592514038085938 seconds =========================
Score: 82
LYYKCGGWHDGKPWTGCTQCVKGAKCVVQNEWY
IYHQCGGSKSAYP-DGSLSCASGSKCVKMNDYY

===================== Banded Alignment (k=0) x400: 0.12568116188049316 seconds =====================
Score: 82
LYYKCGGWHDGKPWTGCTQCVKGAKCVVQNEWY
IYHQCGGSKSAYP-DGSLSCASGSKCVKMNDYY


In [17]:
s1 = "LWGQCGGIAWTGPTTCSPYYSNTPVKCTTYNPYY"
s2 = "EWEQCGGSGGGAVNGVWTGPTCCEVGTTCHYKDQYW"
run_alignments(s1, s2)

======================== Global Alignment x400: 1.3050403594970703 seconds =========================
Score: 48
LWGQCGG-----I-A-WTGPTTCSPYYSNTPVKCTTY-NPYY
EWEQCGGSGGGAVNGVWTGPTCCE-V-GTT---CH-YKDQYW

===================== Banded Alignment (k=0) x400: 0.12092351913452148 seconds =====================
Score: 29
LWGQCGGIAWTGPT-TCS-PYYSNTPVKCTTY-NPYY
EWEQCGGSGGGAVNGVWTGPTCCEVGTTCH-YKDQYW

===================== Banded Alignment (k=1) x400: 0.19289875030517578 seconds =====================
Score: 36
LWGQCGGI---AWTGPTTCSPYYSNTPVKCTTY-NPYY
EWEQCGGSGGGAVNGVWT-GPTCCEVGTTCH-YKDQYW

===================== Banded Alignment (k=2) x400: 0.24329590797424316 seconds =====================
Score: 36
LWGQCGGI---AWTGPTTCSPYYSNTPVKCTTY-NPYY
EWEQCGGSGGGAVNGVWT-GPTCCEVGTTCH-YKDQYW

===================== Banded Alignment (k=3) x400: 0.2993900775909424 seconds ======================
Score: 36
LWGQCGGI---AWTGPTTCSPYYSNTPVKCTTY-NPYY
EWEQCGGSGGGAVNGVWT-GPTCCEVGTTCH-YKDQYW

===================== Banded Alignment (k=4)